# Run protein dynamics with PBCs using PME
## This notebook builds on the previous protein_setup.ipynb and sets up a molecular dynamics simulation using Particle-Mesh Ewald running on CHARMM/OpenMM or CHARMM/BLaDE.
## Note you need to have run the previous notebook and have created the solvated psf and pdb to use this notebook and you need to specify which protein you prepared in the cell below.

In [1]:
# Some choices for pdbids
pdbid = '4pti'  # bovine pancreatic trypsin inhibitor - an x-ray structuture
#pdbid = '6pti'  # bovine pancreatic trypsin inhibitor - an x-ray structuture
#pdbid = '5wyo'  # HDEA a dimeric (two chain) protein - an NMR structure

In [2]:
# This script provides a simple example of running molecular dynamics of 
# a solvated protein structure created in an earlier notebook tutorial
# further illustrating the functionality exposed in pyCHARMM.
#  copyright C.L. Brooks III, June 20, 2022

# These are general python modules needed for this  tutorial
import os
import sys
import numpy as np

# These are a subset of the pycharmm modules that were installed when
# pycharmm was installed in your python environment
import pycharmm
import pycharmm.generate as gen
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dyn
import pycharmm.nbonds as nbonds
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.select as select
import pycharmm.shake as shake

from pycharmm.lib import charmm as libcharmm

## The following are some helper functions for setting-up and running the dynamics

In [3]:
################################
# Ensure that FFT grid is product of small primes 2, 3, 5
def is_factor(n):
    if (n % 2 != 0): return False  # favors even number
    while n:
        flag = False
        for x in (2,3,5):
            if n % x == 0:
               n = n / x
               flag = True
               break

        if flag: continue
        break

    if n == 1: return True
    return False

def checkfft(n, margin = 5):
    n = int(n) + margin
    while 1:
        if is_factor(n): break
        else: n += 1
    return n

In [4]:
#################################
def setup_PBC(boxhalf=0.0, protein_segments=[],solvent_resname='TIP3',ions=[],blade=False):
    """input: boxhalf [0.0]
              solute  []
              solvent_resname ['']
              ions []
              blade [False]
    defines the periodic boundary conditions for a cubic volume of boxsize. 
    Uses: crystal_define_cubic(), crystal.build(), image.setup_residue,
    image.setup_segment to construct symmetry operations. 

    If global variable openmm is true
    the image centering is at [boxhalf,boxhalf,boxhalf] otherwise at [0,0,0].
    """
    crystal.define_cubic(boxhalf*2)
    crystal.build(boxhalf)

    if blade: boxhalf = 0.0 # center at origin for blade
    for segment in protein_segments:
        image.setup_segment(boxhalf,boxhalf, boxhalf, segment)
    if len(solvent_resname)>0: image.setup_residue(boxhalf,boxhalf, boxhalf, solvent_resname)
    for ion in ions:
        image.setup_residue(boxhalf, boxhalf, boxhalf, ion)
    # for systems using centering not at origin, translate coordinates by halfbox
    xyz = coor.get_positions()
    xyz += boxhalf
    coor.set_positions(xyz)
    print(f'Coordinates translated by {boxhalf} A in each dimension to be consistent with image centering')

    return

## Set-up system - topology and parameter files

In [5]:
# Read in the topology (rtf) and parameter file (prm) for proteins
# equivalent to the CHARMM scripting command: read rtf card name toppar/top_all36_prot.rtf
read.rtf('../toppar/top_all36_prot.rtf')
# equivalent to the CHARMM scripting command: read param card flexible name toppar/par_all36m_prot.prm
read.prm('../toppar/par_all36m_prot.prm', flex=True)

# stream in the water/ions parameter using the pycharmm.lingo module
# equivalent to the CHARMM scripting command: stream toppar/toppar_water_ions.str
#pycharmm.lingo.charmm_script('stream ../toppar/toppar_water_ions.str')
read.stream('../toppar/toppar_water_ions.str')

  
 CHARMM>     read rtf card -
 CHARMM>     name ../toppar/top_all36_prot.rtf
 VOPEN> Attempting to open::../toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> MAY 2011 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read param card -
 CHARMM>     name ../toppar/par_all36m_prot.prm -
 CHARMM>     flex
 VOPEN> Attempting to open::../toppar/par_all36m_prot.prm::

          PARAMETER FILE BEING READ FROM UNIT 91
 TITLE> *>>>> CHARMM36 ALL-HYDROGEN PARAMETER FILE FOR PROTEINS <<<<<<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>

## Read in psf and coordinates of solvated systems
## Find the segment names for the solute (first n-2 segments assumed), solvent (penultimate segment assumed) and ions (last segment)

In [6]:
# Read the psf and coordinates for the solvated peptide
# Read psf card name pdb/adp+wat.psf
read.psf_card(f'pdb/{pdbid}+wat.psf')
# read coor pdb name pdb/adp+wat_min.pdb
read.pdb(f'pdb/{pdbid}+wat_min.pdb',resid=True)

  
 CHARMM>     read psf card -
 CHARMM>     name pdb/4pti+wat.psf
 VOPEN> Attempting to open::pdb/4pti+wat.psf::
 MAINIO> Protein structure file being read from unit  91.
 TITLE>  * EXECUTING CHARMM SCRIPT FROM PYTHON
 TITLE>  *  DATE:     8/ 1/24     15:22:12      CREATED BY USER: brookscl
 TITLE>  *
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        3   Number of residues   =     7975
         Number of atoms         =    24640   Number of groups     =     8193
         Number of bonds         =    24648   Number of angles     =     9552
         Number of dihedrals     =     2408   Number of impropers  =      156
         Number of cross-terms   =       58   Number of autogens   =     7911
         Number of HB acceptors  =     7990   Number of HB donors  =    15935
         Number of NB exclusions =        0   Total charge =   -0.00000
 VCLOSE: Closing unit   91 with status "KEEP

## Set-up box size and periodic boundary conditions using the python function setup_PBC

In [7]:
# Now setup periodic boundaries
# boxsize
stats = coor.stat()
xsize = stats['xmax'] - stats['xmin']
ysize = stats['ymax'] - stats['ymin']
zsize = stats['zmax'] - stats['zmin']
boxsize = max(xsize, ysize, zsize)

# half box size
boxhalf = boxsize / 2.0
# Note we could probably do something to extract the information passed to setup_PPC using pyCHARMM functions
# but I didn't have the time so I just made some generic thing
setup_PBC(boxhalf=boxhalf, protein_segments=['PROA', 'PROB', 'PROC'],
          solvent_resname='TIP3',ions=['CLA', 'SOD', 'POT'],blade=False)

 Crystal Parameters : Crystal Type = CUBI
           A     =   64.88500 B    =   64.88500 C     =   64.88500
           Alpha =   90.00000 Beta =   90.00000 Gamma =   90.00000
 XBUILD> Building all transformations with a minimum atom-atom
         contact distance of less than   32.44 Angstroms.

 Range of Grid Search for Transformation     1 :
 Lattice Vector A    -2 TO     2
 Lattice Vector B    -2 TO     2
 Lattice Vector C    -2 TO     2


 The number of transformations generated =    26


 Number  Symop   A   B   C   Distance

      1      1  -1  -1  -1     8.4952
      2      1  -1   0  -1     5.9501
      3      1  -1   1  -1     6.8833
      4      1   0  -1  -1     5.8966
      5      1   0   0  -1     2.6340
      6      1   0   1  -1     4.8901
      7      1  -1  -1   0     4.3919
      8      1  -1   0   0     3.7461
      9      1  -1   1   0     5.0953
     10      1   0  -1   0     2.6464
     11      1   0   1   0     2.6464
     12      1  -1  -1   1     6.6474
     1

## In this section we set-up our nonbonded parameters with cutoff schemes, etc.
## We illustrate two ways to input non-bonded parameters as well as the use of the pyCHARMM _settings.set_warn_level(wrnlev)_ command to alter the warning levels and reduce the output. 
## Note we use two different cutoff schemes (vswitch and vfswitch)

In [8]:
# Set-up non-bonded parameters
# Now specify nonbonded cutoffs for solvated box
cutnb = min(boxhalf,12)
cutim = cutnb
ctofnb = cutnb - 1.0
ctonnb = cutnb - 3.0
# Determine the appropriate cubic fft grid for this boxsize
fft = checkfft(n=np.ceil(boxhalf)*2,margin=0)
# Set-up the parameters
nb_wPME_vsw = pycharmm.NonBondedScript(cutnb=cutnb, cutim=cutim,
                                       ctonnb=ctonnb, ctofnb=ctofnb,
                                       cdie=True, eps=1,
                                       atom=True, vatom=True,
                                       switch=True, vfswitch=False, vswitch=True,
                                       inbfrq=-1, imgfrq=-1,
                                       ewald=True,pmewald=True,kappa=0.32,
                                       fftx=fft,ffty=fft,fftz=fft,order=4)
# Let's set the wrnlev to 0 to avoid the large output
old_wrnlev = settings.set_warn_level(0)
nb_wPME_vsw.run()
settings.set_warn_level(old_wrnlev)
energy.show()
# Let's also set-up a set of nonbonded parameters using vfswitch instead of vswitch
nb_wPME_vfsw_dict = {'cutnb':cutnb, 
                     'cutim':cutim,
                     'ctonnb':ctonnb, 
                     'ctofnb':ctofnb,
                     'cdie':True,
                     'eps':1,
                     'atom':True, 'vatom':True,
                     'switch':True, 'vfswitch':True, 'vswitch':False,
                     'inbfrq':-1, 'imgfrq':-1,
                     'ewald':True,'pmewald':True,'kappa':0.32,
                     'fftx':fft,'ffty':fft,'fftz':fft,'order':4}
# Let's set the wrnlev to 0 to avoid the large output
old_wrnlev = settings.set_warn_level(0)
pycharmm.NonBondedScript(**nb_wPME_vfsw_dict).run()
settings.set_warn_level(old_wrnlev)
energy.show()
# Now go back to the original nonbonded parameters
# Let's set the wrnlev to 0 to avoid the large output
old_wrnlev = settings.set_warn_level(0)
nb_wPME_vsw.run()
settings.set_warn_level(old_wrnlev)
energy.show()

  
 CHARMM>     nbonds cutnb 12 -
 CHARMM>     cutim 12 -
 CHARMM>     ctonnb 9.0 -
 CHARMM>     ctofnb 11.0 -
 CHARMM>     cdie -
 CHARMM>     eps 1 -
 CHARMM>     atom -
 CHARMM>     vatom -
 CHARMM>     switch -
 CHARMM>     vswitch -
 CHARMM>     inbfrq -1 -
 CHARMM>     imgfrq -1 -
 CHARMM>     ewald -
 CHARMM>     pmewald -
 CHARMM>     kappa 0.32 -
 CHARMM>     fftx 72 -
 CHARMM>     ffty 72 -
 CHARMM>     fftz 72 -
 CHARMM>     order 4

 SELECTED IMAGES ATOMS BEING CENTERED ABOUT 32.442500 32.442500 32.442500

 <MKIMAT2>: updating the image atom lists and remapping
 Transformation   Atoms  Groups  Residues  Min-Distance
    1  N1N1N1R1 has      24       8       8        5.79
    2  N1Z0N1R1 has     462     154     154        3.08
    3  N1P1N1R1 has      39      13      13        4.14
    4  Z0N1N1R1 has     384     128     128        2.77
    5  Z0Z0N1R1 has    4099    1367    1367        0.26
    6  Z0P1N1R1 has     534     178     178        0.71
    7  N1N1Z0R1 has     483 


 General atom nonbond list generation found:
  6910653 ATOM PAIRS WERE FOUND FOR ATOM LIST
   461746 GROUP PAIRS REQUIRED ATOM SEARCHES

 SPACE FOR  4367180 ATOM PAIRS AND        0 GROUP PAIRS

 Image nonbond list generation found:
  1139419 ATOM PAIRS WERE FOUND FOR ATOM LIST
        0 ATOM PAIRS WERE FOUND FOR ATOM SELF LIST
   124759 GROUP PAIRS REQUIRED ATOM SEARCHES

  
 CHARMM>     
  

 SELECTED IMAGES ATOMS BEING CENTERED ABOUT 32.442500 32.442500 32.442500

 <MKIMAT2>: updating the image atom lists and remapping
 Transformation   Atoms  Groups  Residues  Min-Distance
    1  N1N1N1R1 has      24       8       8        5.79
    2  N1Z0N1R1 has     462     154     154        3.08
    3  N1P1N1R1 has      39      13      13        4.14
    4  Z0N1N1R1 has     384     128     128        2.77
    5  Z0Z0N1R1 has    4099    1367    1367        0.26
    6  Z0P1N1R1 has     534     178     178        0.71
    7  N1N1Z0R1 has     483     161     161        1.96
    8  N1Z0Z0R1 has    4


 Image nonbond list generation found:
  1139419 ATOM PAIRS WERE FOUND FOR ATOM LIST
        0 ATOM PAIRS WERE FOUND FOR ATOM SELF LIST
   124759 GROUP PAIRS REQUIRED ATOM SEARCHES

ENER ENR:  Eval#     ENERgy      Delta-E         GRMS
ENER INTERN:          BONDs       ANGLes       UREY-b    DIHEdrals    IMPRopers
ENER CROSS:           CMAPs        PMF1D        PMF2D        PRIMO
ENER EXTERN:        VDWaals         ELEC       HBONds          ASP         USER
ENER IMAGES:        IMNBvdw       IMELec       IMHBnd       RXNField    EXTElec
ENER EWALD:          EWKSum       EWSElf       EWEXcl       EWQCor       EWUTil
 ----------       ---------    ---------    ---------    ---------    ---------
ENER>        0 -90270.74399    350.88331      2.15339
ENER INTERN>     2869.86825   2074.59000      8.44369    488.46598      7.02280
ENER CROSS>       -14.82963      0.00000      0.00000      0.00000
ENER EXTERN>     7766.69639 -92863.95807      0.00000      0.00000      0.00000
ENER IMAGES>    

## This function will run a short md start and restart using either CHARMM/OpenMM interface or the CHARMM/BLaDE interface in pyCHARMM

In [9]:
def run_md(useomm=False,useblade=False,nequil=1000,nsteps=5000,nsavc=100,leap=True,lang=True):
    if useomm: append='omm'
    elif useblade: append='blade'
    dyn.set_fbetas(np.full((psf.get_natom()),1.0,dtype=float))
   
    res_file = pycharmm.CharmmFile(file_name=f'res/{pdbid}.res', 
                                   file_unit=2,
                                   formatted=True,read_only=False)
    lam_file = pycharmm.CharmmFile(file_name=f'res/{pdbid}.lam', 
                                   file_unit=3,
                                   formatted=False,read_only=False)
    my_dyn = pycharmm.DynamicsScript(leap=leap, lang=lang, start=True,
                                     nstep=nequil, timest=0.002,
                                     firstt=298.0, finalt=298.0, tbath=298.0,
                                     tstruc=298.0,
                                     teminc=0.0, twindh=0.0, twindl=0.0,
                                     iunwri=res_file.file_unit,
                                     iunlam=lam_file.file_unit,
                                     inbfrq=-1, imgfrq=-1,
                                     iasors=0, iasvel=1, ichecw=0, iscale=0,
                                     iscvel=0,echeck=-1, nsavc=0, nsavv=0, nsavl=0, ntrfrq=0,
                                     isvfrq=nsavc,
                                     iprfrq=2*nsavc, nprint=nsavc, ihtfrq=0, ieqfrq=0,
                                     ilbfrq=0,ihbfrq=0,
                                     omm=useomm, blade=useblade)
    my_dyn.run()

    res_file.close()
    lam_file.close()
    # open unit 2 write form name res/{}.res
    res_file = pycharmm.CharmmFile(file_name=f'res/{pdbid}.res', 
                                   file_unit=2,
                                   formatted=True,read_only=False)
    lam_file = pycharmm.CharmmFile(file_name=f'res/{pdbid}.lam', 
                                   file_unit=3,
                                   formatted=False,read_only=False)
    # open unit 1 write file name dcd/{}.dcd
    dcd_file = pycharmm.CharmmFile(file_name=f'dcd/{pdbid}_{append}.dcd'
                                   ,file_unit=1,
                                   formatted=False,read_only=False)

    my_dyn = pycharmm.DynamicsScript(leap=leap, lang=lang, start=False, restart = True,
                                     nstep=nsteps, timest=0.002,
                                     firstt=298.0, finalt=298.0, tbath=298.0,
                                     tstruc=298.0,
                                     teminc=0.0, twindh=0.0, twindl=0.0,
                                     iunwri=res_file.file_unit,
                                     iunrea=res_file.file_unit,
                                     iuncrd=dcd_file.file_unit,
                                     iunlam=lam_file.file_unit,
                                     inbfrq=-1, imgfrq=-1,
                                     iasors=0, iasvel=1, ichecw=0, iscale=0,
                                     iscvel=0,echeck=-1, nsavc=nsavc, nsavv=0, nsavl=0, ntrfrq=0,
                                     isvfrq=nsavc,
                                     iprfrq=2*nsavc, nprint=nsavc, ihtfrq=0, ieqfrq=0,
                                     ilbfrq=0,ihbfrq=0,
                                     omm=useomm, blade=useblade)
    my_dyn.run()

    res_file.close()
    lam_file.close()
    dcd_file.close()
    return

# Using CHARMM/BLaDE interface through pyCHARMM
## In this section we set-up a short dynamics run equilibration and production using the CHARMM/BLaDE gpu accelerated engine in CHARMM (if a gpu isn't present this should be skipped - we use torch functions to check whether gpu is present).

In [10]:
# Set-up short dynamics
if not os.path.isdir('res'): os.system('mkdir res')
if not os.path.isdir('dcd'): os.system('mkdir dcd')
# Check to see if cuda is available to run BLaDE
import torch
cuda  = torch.cuda.is_available()
num_devices = torch.cuda.device_count()
usedBLaDe = False
if cuda:
    print(f'Running CHARMM/BLaDE MD example on computer with {num_devices} CUDA devices')
    print(f'Running on device {torch.cuda.current_device()} which is a {torch.cuda.get_device_name(torch.cuda.current_device())}')
    run_md(useblade = 'prmc pref 1 iprs 100 prdv 100')
    usedBLaDe = True
else: print('Example not run, no CUDA devices available')

Example not run, no CUDA devices available


# View your trajectory
## Using vmd in your terminal window issue the following command:
> *vmd pdb/4pti+wat.psf dcd/4pti_blade.dcd*

# Using CHARMM/OpenMM interface through pyCHARMM
## In this section we set-up a short dynamics run equilibration using the CHARMM/OpenMM gpu accelerated engine in CHARMM (if a gpu isn't present this should choose the platform _'cpu'_ and run there.

In [11]:
# Set-up short dynamics
if not os.path.isdir('res'): os.system('mkdir res')
if not os.path.isdir('dcd'): os.system('mkdir dcd')
# Check to see if cuda is available to run OpenMM
import torch
cuda  = torch.cuda.is_available()
num_devices = torch.cuda.device_count()
usedOpenMM = False
if cuda and not usedBLaDe:
    print(f'Running CHARMM/OpenMM MD example on computer with {num_devices} CUDA devices')
    print(f'Running on device {torch.cuda.current_device()} which is a {torch.cuda.get_device_name(torch.cuda.current_device())}')
else: print('No CUDA devices available, using either CPU or OpenCL')
if not usedBLaDe: 
    usedOpenMM = True
    run_md(useomm = 'gamma 2 prmc pref 1 iprsfrq 100')

No CUDA devices available, using either CPU or OpenCL
 VOPEN> Attempting to open::res/4pti.res::
 VOPEN> Attempting to open::res/4pti.lam::
  
 CHARMM>     dynamics leap -
 CHARMM>     lang -
 CHARMM>     start -
 CHARMM>     nstep 1000 -
 CHARMM>     timest 0.002 -
 CHARMM>     firstt 298.0 -
 CHARMM>     finalt 298.0 -
 CHARMM>     tbath 298.0 -
 CHARMM>     tstruc 298.0 -
 CHARMM>     teminc 0.0 -
 CHARMM>     twindh 0.0 -
 CHARMM>     twindl 0.0 -
 CHARMM>     iunwri 2 -
 CHARMM>     iunlam 3 -
 CHARMM>     inbfrq -1 -
 CHARMM>     imgfrq -1 -
 CHARMM>     iasors 0 -
 CHARMM>     iasvel 1 -
 CHARMM>     ichecw 0 -
 CHARMM>     iscale 0 -
 CHARMM>     iscvel 0 -
 CHARMM>     echeck -1 -
 CHARMM>     nsavc 0 -
 CHARMM>     nsavv 0 -
 CHARMM>     nsavl 0 -
 CHARMM>     ntrfrq 0 -
 CHARMM>     isvfrq 100 -
 CHARMM>     iprfrq 200 -
 CHARMM>     nprint 100 -
 CHARMM>     ihtfrq 0 -
 CHARMM>     ieqfrq 0 -
 CHARMM>     ilbfrq 0 -
 CHARMM>     ihbfrq 0 -
 CHARMM>     omm gamma 2 prmc pref

 WRIDYN: RESTart file was written at step     200
OpenMM: Velocities scaled or randomized
DYNA>      300      0.60000 -55406.79228  20002.89160 -75409.68388    272.34613
DYNA PROP>          0.00000      0.00000      0.00000      0.00000      0.00000
DYNA INTERN>     9255.36736   4963.78456      0.00000    564.88860     27.32374
DYNA CROSS>       -19.73041      0.00000      0.00000      0.00000
DYNA EXTERN>   -90201.33063      0.00000      0.00000      0.00000      0.00000
DYNA PRESS>         0.00000      0.00000      0.00000      0.00000 273169.95235
 ----------       ---------    ---------    ---------    ---------    ---------
 Crystal Parameters : Crystal Type = CUBI
      DYNA A     =   64.88500 B    =   64.88500 C     =   64.88500
      DYNA Alpha =   90.00000 Beta =   90.00000 Gamma =   90.00000
      DYNA PIXX =        0.00 PIYY =       0.00 PIZZ =       0.00
      DYNA PIXY =        0.00 PIXZ =       0.00 PIYZ =       0.00
      DYNA Gradient Norm =    0.00000

 WRIDYN: RESTart

 WRIDYN: RESTart file was written at step     600
OpenMM: Velocities scaled or randomized
DYNA>      700      1.40000 -52250.60517  21489.56389 -73740.16906    292.58767
DYNA PROP>          0.00000      0.00000      0.00000      0.00000      0.00000
DYNA INTERN>    10191.67513   5252.42834      0.00000    562.59523     29.58718
DYNA CROSS>       -16.04347      0.00000      0.00000      0.00000
DYNA EXTERN>   -89760.42480      0.00000      0.00000      0.00000      0.00000
DYNA PRESS>         0.00000      0.00000      0.00000      0.00000 273127.02347
 ----------       ---------    ---------    ---------    ---------    ---------
 Crystal Parameters : Crystal Type = CUBI
      DYNA A     =   64.88160 B    =   64.88160 C     =   64.88160
      DYNA Alpha =   90.00000 Beta =   90.00000 Gamma =   90.00000
      DYNA PIXX =        0.00 PIYY =       0.00 PIZZ =       0.00
      DYNA PIXY =        0.00 PIXZ =       0.00 PIYZ =       0.00
      DYNA Gradient Norm =    0.00000

 WRIDYN: RESTart

 WRIDYN: RESTart file was written at step    1000
 VCLOSE: Closing unit    2 with status "KEEP"

     DETAILS ABOUT CENTRE OF MASS
     POSITION          :    32.306474        32.220420        32.279080    
     VELOCITY          :   4.28448316E-04   1.62184701E-03   2.56341828E-03
     ANGULAR MOMENTUM  :    434.70533       -6107.9909       -13390.509    
     KINETIC ENERGY    :   0.70061979    
  
 CHARMM>     
  
 VCLOSE: Closing unit    2 with status "KEEP"
 VCLOSE: Closing unit    3 with status "KEEP"
 VOPEN> Attempting to open::res/4pti.res::
 VOPEN> Attempting to open::res/4pti.lam::
 VOPEN> Attempting to open::dcd/4pti_omm.dcd::
  
 CHARMM>     dynamics leap -
 CHARMM>     lang -
 CHARMM>     restart -
 CHARMM>     nstep 5000 -
 CHARMM>     timest 0.002 -
 CHARMM>     firstt 298.0 -
 CHARMM>     finalt 298.0 -
 CHARMM>     tbath 298.0 -
 CHARMM>     tstruc 298.0 -
 CHARMM>     teminc 0.0 -
 CHARMM>     twindh 0.0 -
 CHARMM>     twindl 0.0 -
 CHARMM>     iunwri 2 -
 CHARMM>    

 WRIDYN: RESTart file was written at step    1200
DYNA>      300      2.60000 -51137.44469  22059.94235 -73197.38703    300.35357
DYNA PROP>          0.00000      0.00000      0.00000      0.00000      0.00000
DYNA INTERN>    10315.74712   5315.89369      0.00000    557.23271     33.31518
DYNA CROSS>       -15.71281      0.00000      0.00000      0.00000
DYNA EXTERN>   -89403.79836      0.00000      0.00000      0.00000      0.00000
DYNA PRESS>         0.00000      0.00000      0.00000      0.00000 273127.02347
 ----------       ---------    ---------    ---------    ---------    ---------
 Crystal Parameters : Crystal Type = CUBI
      DYNA A     =   64.88160 B    =   64.88160 C     =   64.88160
      DYNA Alpha =   90.00000 Beta =   90.00000 Gamma =   90.00000
      DYNA PIXX =        0.00 PIYY =       0.00 PIZZ =       0.00
      DYNA PIXY =        0.00 PIXZ =       0.00 PIYZ =       0.00
      DYNA Gradient Norm =    0.00000

 WRIDYN: RESTart file was written at step    1300
DYNA> 

...

 WRIDYN: RESTart file was written at step    4400
DYNA>     3500      9.00000 -52176.06771  21949.97611 -74126.04382    298.85634
DYNA PROP>          0.00000      0.00000      0.00000      0.00000      0.00000
DYNA INTERN>    10590.90247   5344.33024      0.00000    559.13291     34.03520
DYNA CROSS>       -19.04815      0.00000      0.00000      0.00000
DYNA EXTERN>   -90635.36504      0.00000      0.00000      0.00000      0.00000
DYNA PRESS>         0.00000      0.00000      0.00000      0.00000 258543.48049
 ----------       ---------    ---------    ---------    ---------    ---------
 Crystal Parameters : Crystal Type = CUBI
      DYNA A     =   63.70564 B    =   63.70564 C     =   63.70564
      DYNA Alpha =   90.00000 Beta =   90.00000 Gamma =   90.00000
      DYNA PIXX =        0.00 PIYY =       0.00 PIZZ =       0.00
      DYNA PIXY =        0.00 PIXZ =       0.00 PIYZ =       0.00
      DYNA Gradient Norm =    0.00000

 WRIDYN: RESTart file was written at step    4500
DYNA> 

 WRIDYN: RESTart file was written at step    4800
DYNA>     3900      9.80000 -52340.45207  21802.97942 -74143.43149    296.85493
DYNA PROP>          0.00000      0.00000      0.00000      0.00000      0.00000
DYNA INTERN>    10638.34044   5443.77577      0.00000    552.47996     27.68029
DYNA CROSS>       -10.12301      0.00000      0.00000      0.00000
DYNA EXTERN>   -90795.58852      0.00000      0.00000      0.00000      0.00000
DYNA PRESS>         0.00000      0.00000      0.00000      0.00000 257712.28153
 ----------       ---------    ---------    ---------    ---------    ---------
 Crystal Parameters : Crystal Type = CUBI
      DYNA A     =   63.63729 B    =   63.63729 C     =   63.63729
      DYNA Alpha =   90.00000 Beta =   90.00000 Gamma =   90.00000
      DYNA PIXX =        0.00 PIYY =       0.00 PIZZ =       0.00
      DYNA PIXY =        0.00 PIXZ =       0.00 PIYZ =       0.00
      DYNA Gradient Norm =    0.00000

 WRIDYN: RESTart file was written at step    4900
DYNA> 

 WRIDYN: RESTart file was written at step    5200
DYNA>     4300     10.60000 -52424.58953  21977.56567 -74402.15520    299.23198
DYNA PROP>          0.00000      0.00000      0.00000      0.00000      0.00000
DYNA INTERN>    10676.69713   5423.81085      0.00000    557.29579     32.12268
DYNA CROSS>       -20.65759      0.00000      0.00000      0.00000
DYNA EXTERN>   -91071.43101      0.00000      0.00000      0.00000      0.00000
DYNA PRESS>         0.00000      0.00000      0.00000      0.00000 252585.79100
 ----------       ---------    ---------    ---------    ---------    ---------
 Crystal Parameters : Crystal Type = CUBI
      DYNA A     =   63.21250 B    =   63.21250 C     =   63.21250
      DYNA Alpha =   90.00000 Beta =   90.00000 Gamma =   90.00000
      DYNA PIXX =        0.00 PIYY =       0.00 PIZZ =       0.00
      DYNA PIXY =        0.00 PIXZ =       0.00 PIYZ =       0.00
      DYNA Gradient Norm =    0.00000

 WRIDYN: RESTart file was written at step    5300
DYNA> 

 WRIDYN: RESTart file was written at step    5600
DYNA>     4700     11.40000 -52727.63877  22118.05133 -74845.69009    301.14474
DYNA PROP>          0.00000      0.00000      0.00000      0.00000      0.00000
DYNA INTERN>    10526.25049   5453.31359      0.00000    559.70493     32.54854
DYNA CROSS>       -16.54525      0.00000      0.00000      0.00000
DYNA EXTERN>   -91400.96017      0.00000      0.00000      0.00000      0.00000
DYNA PRESS>         0.00000      0.00000      0.00000      0.00000 249416.97759
 ----------       ---------    ---------    ---------    ---------    ---------
 Crystal Parameters : Crystal Type = CUBI
      DYNA A     =   62.94704 B    =   62.94704 C     =   62.94704
      DYNA Alpha =   90.00000 Beta =   90.00000 Gamma =   90.00000
      DYNA PIXX =        0.00 PIYY =       0.00 PIZZ =       0.00
      DYNA PIXY =        0.00 PIXZ =       0.00 PIYZ =       0.00
      DYNA Gradient Norm =    0.00000

 WRIDYN: RESTart file was written at step    5700
DYNA> 

 WRIDYN: RESTart file was written at step    6000
 VCLOSE: Closing unit    2 with status "KEEP"

     DETAILS ABOUT CENTRE OF MASS
     POSITION          :    31.444481        31.257468        31.554550    
     VELOCITY          :   1.52537968E-04   1.74792090E-03   3.46756877E-05
     ANGULAR MOMENTUM  :   -7743.4256        14328.261       -11960.776    
     KINETIC ENERGY    :   0.22990744    
  
 CHARMM>     
  
 VCLOSE: Closing unit    2 with status "KEEP"
 VCLOSE: Closing unit    3 with status "KEEP"
 VCLOSE: Closing unit    1 with status "KEEP"


# View your trajectory
## Here we use NGLView and mdtraj

In [13]:
import nglview as nv
import mdtraj as md
view = nv.NGLWidget()
if usedBLaDe: trajectory = f'dcd/{pdbid}_blade.dcd'
if usedOpenMM: trajectory = f'dcd/{pdbid}_omm.dcd'
if (usedBLaDe or usedOpenMM):
    traj = md.load(trajectory,top=f'pdb/{pdbid}+wat.psf')
    t = nv.adaptor.MDTrajTrajectory(traj)
    view = nv.NGLWidget(t)
    view.clear_representations()
    view.add_representation('ball+stick',selection='protein')
    view.add_representation('cartoon',selection='protein',color_scheme='resname')
    view.add_representation('licorice',selection='water',opacity=0.4)
    view.add_representation('spacefill',selection='CLA',color='green')
view

NGLWidget(max_frame=49)